In [61]:
# simulate the timestamp for each activity
# read the data
import pandas as pd

year = '2020'
train_data_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data/bpic_{year}_gt/autonomous/train_preprocessed.csv"
train_data = pd.read_csv(train_data_path)

In [62]:
# get the work time period for each agent
def get_agent_working_scope(train_data):

    # ensure timestamp is datetime
    df = train_data.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # extract weekday (0=Mon, 6=Sun) and time
    df['weekday'] = df['timestamp'].dt.weekday
    df['time'] = df['timestamp'].dt.time

    scope = {}

    # group by agent
    for agent, agent_df in df.groupby('agent'):

        agent_scope = {}

        # group by weekday for this agent
        for wd, day_df in agent_df.groupby('weekday'):
            start_time = day_df['time'].min()
            end_time = day_df['time'].max()

            # convert to HH:MM:SS string
            agent_scope[wd] = (start_time.strftime('%H:%M:%S'),
                               end_time.strftime('%H:%M:%S'))

        scope[agent] = agent_scope

    return scope


time_scope = get_agent_working_scope(train_data)
print(time_scope)

{0: {0: ('00:03:13', '23:59:52'), 1: ('00:00:02', '23:59:47'), 2: ('00:00:14', '23:59:29'), 3: ('00:00:07', '23:59:44'), 4: ('00:00:07', '23:59:26'), 5: ('00:00:01', '23:58:51'), 6: ('00:01:39', '23:58:16')}, 1: {0: ('01:03:15', '23:59:50'), 1: ('00:00:02', '23:59:42'), 2: ('00:00:12', '23:59:52'), 3: ('00:00:12', '23:59:59'), 4: ('00:00:24', '23:58:16'), 5: ('00:00:56', '23:45:00'), 6: ('00:32:51', '23:53:08')}, 2: {0: ('00:03:15', '23:57:08'), 1: ('00:00:38', '23:58:54'), 2: ('00:00:30', '23:59:25'), 3: ('00:05:56', '23:59:50'), 4: ('00:02:29', '23:53:47'), 5: ('00:00:35', '23:55:24'), 6: ('00:08:50', '23:37:22')}, 3: {0: ('02:22:36', '23:58:11'), 1: ('00:26:01', '23:46:52'), 2: ('00:05:31', '23:55:39'), 3: ('00:06:07', '23:44:32'), 4: ('00:07:25', '23:55:59'), 5: ('00:26:25', '22:59:31'), 6: ('00:11:21', '23:19:35')}, 4: {0: ('17:17:50', '23:51:54'), 1: ('00:23:55', '23:56:27'), 2: ('00:38:41', '23:49:43'), 3: ('00:00:32', '23:39:40'), 4: ('00:13:09', '23:49:50'), 5: ('00:01:14', '0

In [63]:
# for each agent, the period of switch of one pair of activities should follow a distribution in the same case
# normal distribution
import pandas as pd
from collections import defaultdict
import numpy as np


def get_agent_activity_pair_distributions(train_data):
    df = train_data.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # ensure proper ordering
    df = df.sort_values(['case_id', 'timestamp'])

    # dictionary: agent -> (A, B) -> list of durations (seconds)
    durations = defaultdict(lambda: defaultdict(list))

    # group by trace
    for _, trace_df in df.groupby('case_id'):
        events = trace_df[['agent', 'activity_name', 'timestamp']].values

        # iterate consecutive events
        for i in range(len(events) - 1):
            agent_A, act_A, ts_A = events[i]
            agent_B, act_B, ts_B = events[i + 1]

            # only consider pairs triggered by the same agent
            if agent_A != agent_B:
                continue

            delta = (ts_B - ts_A).total_seconds()
            # if delta < 0:
            #     continue  # safety check

            durations[agent_A][(act_A, act_B)].append(delta)

    # now fit normal distributions
    results = {}

    for agent, pair_dict in durations.items():
        agent_result = {}

        for pair, values in pair_dict.items():
            if len(values) >= 1:
                mu = float(np.mean(values))
                sigma = float(np.std(values, ddof=1)) if len(values) > 1 else 0.0

                agent_result[pair] = {
                    "mean": round(mu, 3),
                    "std": round(sigma, 3),
                    "count": len(values)
                }

        results[agent] = agent_result

    return results


agent_act_pair_duration = get_agent_activity_pair_distributions(train_data)
agent_act_pair_duration

{0: {('Start trip', 'End trip'): {'mean': 456310.445,
   'std': 663937.108,
   'count': 4739},
  ('End trip', 'Declaration SUBMITTED'): {'mean': 1003660.063,
   'std': 1639544.404,
   'count': 3673},
  ('Permit REJECTED', 'Start trip'): {'mean': 3320686.595,
   'std': 3159676.052,
   'count': 42},
  ('Declaration REJECTED', 'Declaration SUBMITTED'): {'mean': 211710.112,
   'std': 562940.725,
   'count': 1324},
  ('Permit REJECTED', 'Permit SUBMITTED'): {'mean': 358878.598,
   'std': 947502.501,
   'count': 194},
  ('Request For Payment REJECTED',
   'Request For Payment SUBMITTED'): {'mean': 177566.015, 'std': 452605.797, 'count': 130},
  ('Start trip', 'Declaration SUBMITTED'): {'mean': 961730.259,
   'std': 1568022.439,
   'count': 166},
  ('Declaration SUBMITTED', 'End trip'): {'mean': 179489.681,
   'std': 180361.771,
   'count': 47},
  ('Request For Payment SUBMITTED',
   'Request For Payment SUBMITTED'): {'mean': 325770.929, 'std': 839824.117, 'count': 14},
  ('Permit SAVED', 'St

In [64]:
# for each agent, the period of switch of one pair of activities should follow a distribution in different cases
# normal distribution

def get_agent_cross_case_switch_distributions(train_data):

    df = train_data.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    # Sort globally by agent then time
    df = df.sort_values(["agent", "timestamp"])

    # Storage: agent -> (A,B) -> list of durations (seconds)
    durations = defaultdict(lambda: defaultdict(list))

    # Group by agent
    for agent, agent_df in df.groupby("agent"):

        events = agent_df[["case_id", "activity_name", "timestamp"]].values

        for i in range(len(events) - 1):
            case_A, act_A, ts_A = events[i]
            case_B, act_B, ts_B = events[i + 1]

            # Only consider cross-case transitions
            if case_A == case_B:
                continue
            
            delta = (ts_B - ts_A).total_seconds()
            # if delta < 0:
            #     continue  # safety

            durations[agent][(act_A, act_B)].append(delta)

    # Fit normal distributions
    results = {}

    for agent, pair_dict in durations.items():
        agent_result = {}

        for pair, vals in pair_dict.items():
            if len(vals) >= 1:
                mu = float(np.mean(vals))
                std = float(np.std(vals, ddof=1)) if len(vals) > 1 else 0.0

                agent_result[pair] = {
                    "mean": round(mu, 3),
                    "std": round(std, 3),
                    "count": len(vals)
                }

        results[agent] = agent_result

    return results


agent_diff_case_act_duration = get_agent_cross_case_switch_distributions(train_data)
agent_diff_case_act_duration

{0: {('End trip', 'Start trip'): {'mean': 3187.408,
   'std': 93310.92,
   'count': 2033},
  ('Start trip', 'End trip'): {'mean': 1671.429,
   'std': 38794.532,
   'count': 2016},
  ('Start trip', 'Start trip'): {'mean': 620.405,
   'std': 9344.783,
   'count': 3058},
  ('Start trip', 'Permit SUBMITTED'): {'mean': 30178.945,
   'std': 15447.798,
   'count': 110},
  ('Permit SUBMITTED', 'Permit SUBMITTED'): {'mean': 1792.951,
   'std': 3614.863,
   'count': 2621},
  ('Permit SUBMITTED', 'Start trip'): {'mean': 14917.077,
   'std': 10642.846,
   'count': 130},
  ('Permit SUBMITTED', 'End trip'): {'mean': 14247.93,
   'std': 10921.491,
   'count': 128},
  ('End trip', 'End trip'): {'mean': 170.526, 'std': 3835.262, 'count': 3040},
  ('Start trip', 'Permit REJECTED'): {'mean': 35094.5,
   'std': 4069.839,
   'count': 4},
  ('Permit SUBMITTED', 'Request For Payment SUBMITTED'): {'mean': 1890.921,
   'std': 4256.459,
   'count': 533},
  ('Request For Payment SUBMITTED', 'Declaration SUBMITTE

In [65]:
'''
def simulate_event_timestamps(df_events, same_case_dists, cross_case_dists, working_scope, global_start_time):

    df = df_events.copy()

    # Remove any existing timestamps
    for col in ["start_timestamp", "end_timestamp"]:
        if col in df.columns:
            df = df.drop(columns=[col])

    # Sort by case to preserve case order
    # df = df.sort_values(["case_id"]).reset_index(drop=True)

    # Output columns
    df["start_timestamp"] = None
    df["end_timestamp"] = None

    # Track last timestamp, case, and activity per agent
    agent_last_timestamp = {}
    agent_last_case = {}
    agent_last_act = {}

    # Sample from normal
    def sample_normal(mu, sigma):
        if sigma == 0:
            return mu
        return max(1, np.random.normal(mu, sigma))

    # Align timestamp to next available working time
    def align_to_working_scope(agent, ts):
        if agent not in working_scope:
            return ts

        weekday = ts.weekday()

        valid_weekdays = sorted(working_scope[agent].keys())

        # If today is not valid → jump forward
        if weekday not in valid_weekdays:
            next_wd = valid_weekdays[0]
            days_forward = (next_wd - weekday) % 7
            next_ts = ts + pd.Timedelta(days=days_forward)
            start_str = working_scope[agent][next_wd][0]
            hh, mm, ss = map(int, start_str.split(":"))
            return next_ts.replace(hour=hh, minute=mm, second=ss)

        # Today is valid: align time
        start_str, end_str = working_scope[agent][weekday]
        sh, sm, ss = map(int, start_str.split(":"))
        eh, em, es = map(int, end_str.split(":"))

        start_t = ts.replace(hour=sh, minute=sm, second=ss)
        end_t   = ts.replace(hour=eh, minute=em, second=es)

        if ts < start_t:
            return start_t
        if ts > end_t:
            # move to next day
            next_ts = ts + pd.Timedelta(days=1)
            next_wd = next_ts.weekday()
            if next_wd in working_scope[agent]:
                ns, = [working_scope[agent][next_wd][0]]
                hh, mm, ss = map(int, ns.split(":"))
                return next_ts.replace(hour=hh, minute=mm, second=ss)
            return next_ts

        return ts

    # Simulate per case
    for case_id, case_df in df.groupby("case_id", sort=False):

        for idx, row in case_df.iterrows():
            act = row["activity_name"]
            agent = row["resource"]

            # FIRST event for this agent
            if agent not in agent_last_timestamp:

                # If agent has working scope: start at that weekday/time
                if agent in working_scope:
                    first_wd = sorted(working_scope[agent].keys())[0]
                    start_str = working_scope[agent][first_wd][0]
                    hh, mm, ss = map(int, start_str.split(":"))

                    ts = global_start_time + pd.Timedelta(days=first_wd)
                    ts = ts.replace(hour=hh, minute=mm, second=ss)

                else:
                    ts = global_start_time

            else:
                last_ts = agent_last_timestamp[agent]
                last_case = agent_last_case[agent]
                last_act = agent_last_act[agent]

                # SAME CASE
                if last_case == case_id:
                    key = (last_act, act)
                    if agent in same_case_dists and key in same_case_dists[agent]:
                        mu = same_case_dists[agent][key]["mean"]
                        sigma = same_case_dists[agent][key]["std"]
                        delta = sample_normal(mu, sigma)
                    else:
                        delta = 60  # fallback: 1 min

                # CROSS CASE
                else:
                    key = (last_act, act)
                    if agent in cross_case_dists and key in cross_case_dists[agent]:
                        mu = cross_case_dists[agent][key]["mean"]
                        sigma = cross_case_dists[agent][key]["std"]
                        delta = sample_normal(mu, sigma)
                    else:
                        delta = 120  # fallback: 2 min

                ts = last_ts + pd.Timedelta(seconds=delta)

            # Align to working scope
            ts = align_to_working_scope(agent, ts)

            # Assign start & end timestamps
            df.at[idx, "start_timestamp"] = ts.floor('s')
            df.at[idx, "end_timestamp"] = (ts + pd.Timedelta(minutes=10)).floor('s')

            # Update memory
            agent_last_timestamp[agent] = ts.floor('s')
            agent_last_case[agent] = case_id
            agent_last_act[agent] = act

    return df


log_l = [0,1,2,3,4,5,6,7,8,9]
train_split_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data/bpic_{year}_gt/autonomous/train_preprocessed.csv"
train_split_data = pd.read_csv(train_split_path)
for i in log_l:
    prob_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_prob/autonomous/simulated_log_{i}.csv"
    prob_data = pd.read_csv(prob_path)
    train_split_data['timestamp'] = pd.to_datetime(train_split_data['timestamp'])
    first_resource = prob_data.iloc[0]['resource']
    resource_data = train_split_data[train_split_data['resource'] == first_resource]
    last_active_time = resource_data['timestamp'].max()
    prob_df_with_time = simulate_event_timestamps(prob_data, agent_act_pair_duration, agent_diff_case_act_duration, time_scope, last_active_time)
    prob_output_file_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_prob_time/autonomous/simulated_log_{i}.csv"
    prob_df_with_time.to_csv(prob_output_file_path, index=False)

for j in log_l:
    petri_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_petri/autonomous/simulated_log_{j}.csv"
    petri_data = pd.read_csv(petri_path)
    train_split_data['timestamp'] = pd.to_datetime(train_split_data['timestamp'])
    first_resource = petri_data.iloc[0]['resource']
    resource_data = train_split_data[train_split_data['resource'] == first_resource]
    last_active_time = resource_data['timestamp'].max()
    petri_df_with_time = simulate_event_timestamps(petri_data, agent_act_pair_duration, agent_diff_case_act_duration, time_scope, last_active_time)
    petri_output_file_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_petri_time/autonomous/simulated_log_{j}.csv"
    petri_df_with_time.to_csv(petri_output_file_path, index=False)
'''

'\ndef simulate_event_timestamps(df_events, same_case_dists, cross_case_dists, working_scope, global_start_time):\n\n    df = df_events.copy()\n\n    # Remove any existing timestamps\n    for col in ["start_timestamp", "end_timestamp"]:\n        if col in df.columns:\n            df = df.drop(columns=[col])\n\n    # Sort by case to preserve case order\n    # df = df.sort_values(["case_id"]).reset_index(drop=True)\n\n    # Output columns\n    df["start_timestamp"] = None\n    df["end_timestamp"] = None\n\n    # Track last timestamp, case, and activity per agent\n    agent_last_timestamp = {}\n    agent_last_case = {}\n    agent_last_act = {}\n\n    # Sample from normal\n    def sample_normal(mu, sigma):\n        if sigma == 0:\n            return mu\n        return max(1, np.random.normal(mu, sigma))\n\n    # Align timestamp to next available working time\n    def align_to_working_scope(agent, ts):\n        if agent not in working_scope:\n            return ts\n\n        weekday = ts.we

In [66]:
'''
# another approach
import pandas as pd
import numpy as np
from collections import defaultdict

def simulate_realistic_timestamps(events_df, train_data, agent_scope):
    """
    Simulate timestamps preserving:
    - agent working-time scope
    - real consecutive activity durations
    - realistic absolute timeline (reduce AEDD)
    
    Parameters
    ----------
    events_df : pd.DataFrame
        Must have 'case_id', 'activity_name', 'resource'
    train_data : pd.DataFrame
        Must have 'case_id', 'activity_type', 'agent_id', 'timestamp'
    agent_scope : dict
        agent -> weekday -> (start_time, end_time)
    
    Returns
    -------
    pd.DataFrame with 'predicted_start' and 'predicted_end'
    """
    df = events_df.copy()
    df['start_timestamp'] = None
    df['end_timestamp'] = None
    
    train_data['timestamp'] = pd.to_datetime(train_data['timestamp'])
    pair_durations = defaultdict(lambda: defaultdict(list))
    
    for agent, agent_df in train_data.groupby('agent_id'):
        agent_df = agent_df.sort_values('timestamp')
        events = agent_df[['case_id', 'activity_type', 'timestamp']].values
        for i in range(len(events)-1):
            act1, act2 = events[i][1], events[i+1][1]
            t1, t2 = events[i][2], events[i+1][2]
            delta = (t2 - t1).total_seconds()
            # if delta > 0:
            pair_durations[agent][(act1, act2)].append(delta)
    
    # Compute mean/std for each pair
    pair_stats = {}
    for agent, pairs in pair_durations.items():
        agent_res = {}
        for pair, vals in pairs.items():
            vals = np.array(vals)
            # Clip extreme values to 5-95 percentile
            vals = np.clip(vals, np.percentile(vals,30), np.percentile(vals,70))
            mu, sigma = np.mean(vals), np.std(vals, ddof=1)
            agent_res[pair] = {'mean': mu, 'std': sigma}
        pair_stats[agent] = agent_res
    
    last_real_ts = train_data.groupby('agent_id')['timestamp'].max().to_dict()
    
    last_ts = {}
    last_act = {}
    
    for idx, row in df.iterrows():
        agent = row['resource']
        act = row['activity_name']
        
        # Initialize timestamp from last real timestamp or 1 Jan 2025
        if agent not in last_ts:
            ts = last_real_ts.get(agent, pd.Timestamp("2025-01-01 09:00:00"))
        else:
            prev_act = last_act[agent]
            prev_ts = last_ts[agent]
            # Sample duration
            if agent in pair_stats and (prev_act, act) in pair_stats[agent]:
                mu = pair_stats[agent][(prev_act, act)]['mean']
                sigma = pair_stats[agent][(prev_act, act)]['std']
                delta = max(1, np.random.normal(mu, sigma))
            else:
                delta = 60
            ts = prev_ts + pd.Timedelta(seconds=delta)
        
        # Align to agent working time
        weekday = ts.weekday()
        if agent in agent_scope and weekday in agent_scope[agent]:
            start_str, end_str = agent_scope[agent][weekday]
            sh, sm, ss = map(int, start_str.split(':'))
            eh, em, es = map(int, end_str.split(':'))
            start_t = ts.replace(hour=sh, minute=sm, second=ss)
            end_t   = ts.replace(hour=eh, minute=em, second=es)
            if ts < start_t:
                ts = start_t
            elif ts > end_t:
                ts += pd.Timedelta(days=1)
                ts = ts.replace(hour=sh, minute=sm, second=ss)
        
        df.at[idx, 'start_timestamp'] = ts.floor('s')
        df.at[idx, 'end_timestamp']   = (ts + pd.Timedelta(seconds=1)).floor('s')
        
        last_ts[agent] = ts
        last_act[agent] = act
    
    return df


log_l = [0,1,2,3,4,5,6,7,8,9]
train_split_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data/bpic_{year}_gt/autonomous/train_preprocessed.csv"
train_split_data = pd.read_csv(train_split_path)
for i in log_l:
    prob_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_prob/autonomous/simulated_log_{i}.csv"
    prob_data = pd.read_csv(prob_path)
    train_split_data['timestamp'] = pd.to_datetime(train_split_data['timestamp'])
    first_resource = prob_data.iloc[0]['resource']
    resource_data = train_split_data[train_split_data['resource'] == first_resource]
    last_active_time = resource_data['timestamp'].max()
    prob_df_with_time = simulate_realistic_timestamps(prob_data, train_data, time_scope)
    prob_output_file_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_prob_time/autonomous/simulated_log_{i}.csv"
    prob_df_with_time.to_csv(prob_output_file_path, index=False)

for j in log_l:
    petri_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_petri/autonomous/simulated_log_{j}.csv"
    petri_data = pd.read_csv(petri_path)
    train_split_data['timestamp'] = pd.to_datetime(train_split_data['timestamp'])
    first_resource = petri_data.iloc[0]['resource']
    resource_data = train_split_data[train_split_data['resource'] == first_resource]
    last_active_time = resource_data['timestamp'].max()
    petri_df_with_time = simulate_realistic_timestamps(petri_data, train_data, time_scope)
    petri_output_file_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_petri_time/autonomous/simulated_log_{j}.csv"
    petri_df_with_time.to_csv(petri_output_file_path, index=False)
'''

'\n# another approach\nimport pandas as pd\nimport numpy as np\nfrom collections import defaultdict\n\ndef simulate_realistic_timestamps(events_df, train_data, agent_scope):\n    """\n    Simulate timestamps preserving:\n    - agent working-time scope\n    - real consecutive activity durations\n    - realistic absolute timeline (reduce AEDD)\n    \n    Parameters\n    ----------\n    events_df : pd.DataFrame\n        Must have \'case_id\', \'activity_name\', \'resource\'\n    train_data : pd.DataFrame\n        Must have \'case_id\', \'activity_type\', \'agent_id\', \'timestamp\'\n    agent_scope : dict\n        agent -> weekday -> (start_time, end_time)\n    \n    Returns\n    -------\n    pd.DataFrame with \'predicted_start\' and \'predicted_end\'\n    """\n    df = events_df.copy()\n    df[\'start_timestamp\'] = None\n    df[\'end_timestamp\'] = None\n    \n    train_data[\'timestamp\'] = pd.to_datetime(train_data[\'timestamp\'])\n    pair_durations = defaultdict(lambda: defaultdict(

In [67]:
''''
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import gamma

def simulate_realistic_timestamps_gamma(events_df, train_data, agent_scope,
                                        max_delta_seconds=43200):
    """
    Simulate timestamps using Gamma distribution for consecutive activity durations,
    with robust safeguards to avoid invalid Gamma parameters and overflow.
    
    Parameters
    ----------
    events_df : pd.DataFrame
        Must have 'case_id', 'activity_name', 'resource'
    train_data : pd.DataFrame
        Must have 'case_id', 'activity_type', 'agent_id', 'timestamp'
    agent_scope : dict
        agent -> weekday -> (start_time, end_time)
    max_delta_seconds : float
        Maximum allowed duration in seconds to prevent overflow (default 12 hours)
    
    Returns
    -------
    pd.DataFrame with 'start_timestamp' and 'end_timestamp'
    """
    df = events_df.copy()
    df['start_timestamp'] = None
    df['end_timestamp'] = None
    
    # Ensure timestamps
    train_data['timestamp'] = pd.to_datetime(train_data['timestamp'])
    
    # Collect consecutive activity durations
    pair_durations = defaultdict(lambda: defaultdict(list))
    for agent, agent_df in train_data.groupby('agent_id'):
        agent_df = agent_df.sort_values('timestamp')
        events = agent_df[['case_id', 'activity_type', 'timestamp']].values
        for i in range(len(events)-1):
            act1, act2 = events[i][1], events[i+1][1]
            t1, t2 = events[i][2], events[i+1][2]
            delta = (t2 - t1).total_seconds()
            if delta > 0:
                pair_durations[agent][(act1, act2)].append(delta)
    
    # Fit Gamma parameters with robust safeguards
    pair_stats = {}
    for agent, pairs in pair_durations.items():
        agent_res = {}
        for pair, vals in pairs.items():
            vals = np.array(vals)
            if len(vals) == 0:
                continue
            # Clip extremes if more than 1 observation
            if len(vals) > 1:
                vals = np.clip(vals, np.percentile(vals,30), np.percentile(vals,70))
            
            mean = np.mean(vals)
            var  = np.var(vals, ddof=1) if len(vals) > 1 else mean*0.1  # small positive var for single obs
            
            # Compute shape and scale safely
            shape = max(mean**2 / var, 1e-3)
            scale = max(var / mean, 1e-3)
            
            agent_res[pair] = {'shape': shape, 'scale': scale}
        pair_stats[agent] = agent_res
    
    # Get last real timestamp per agent
    last_real_ts = train_data.groupby('agent_id')['timestamp'].max().to_dict()
    
    last_ts = {}
    last_act = {}
    
    for idx, row in df.iterrows():
        agent = row['resource']
        act = row['activity_name']
        
        # Initialize timestamp from last real timestamp or fallback
        if agent not in last_ts:
            ts = last_real_ts.get(agent, pd.Timestamp("2025-01-01 09:00:00"))
        else:
            prev_act = last_act[agent]
            prev_ts = last_ts[agent]
            
            # Sample duration from Gamma
            if agent in pair_stats and (prev_act, act) in pair_stats[agent]:
                shape = max(pair_stats[agent][(prev_act, act)]['shape'], 1e-3)
                scale = max(pair_stats[agent][(prev_act, act)]['scale'], 1e-3)
                delta = gamma.rvs(shape, scale=scale)
                # Safeguard: clip delta to reasonable range
                delta = max(1, min(delta, max_delta_seconds))
            else:
                delta = 60  # fallback 1 second
            
            ts = prev_ts + pd.Timedelta(seconds=delta)
        
        # Align to agent working time
        weekday = ts.weekday()
        if agent in agent_scope and weekday in agent_scope[agent]:
            start_str, end_str = agent_scope[agent][weekday]
            sh, sm, ss = map(int, start_str.split(':'))
            eh, em, es = map(int, end_str.split(':'))
            start_t = ts.replace(hour=sh, minute=sm, second=ss)
            end_t   = ts.replace(hour=eh, minute=em, second=es)
            if ts < start_t:
                ts = start_t
            elif ts > end_t:
                ts += pd.Timedelta(days=1)
                ts = ts.replace(hour=sh, minute=sm, second=ss)
        
        # Assign timestamps rounded to seconds
        df.at[idx, 'start_timestamp'] = ts.floor('s')
        df.at[idx, 'end_timestamp']   = (ts + pd.Timedelta(seconds=1)).floor('s')
        
        last_ts[agent] = ts
        last_act[agent] = act
    
    return df


log_l = [0,1,2,3,4,5,6,7,8,9]
train_split_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data/bpic_{year}_gt/autonomous/train_preprocessed.csv"
train_split_data = pd.read_csv(train_split_path)
for i in log_l:
    prob_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_prob/autonomous/simulated_log_{i}.csv"
    prob_data = pd.read_csv(prob_path)
    train_split_data['timestamp'] = pd.to_datetime(train_split_data['timestamp'])
    first_resource = prob_data.iloc[0]['resource']
    resource_data = train_split_data[train_split_data['resource'] == first_resource]
    last_active_time = resource_data['timestamp'].max()
    prob_df_with_time = simulate_realistic_timestamps_gamma(prob_data, train_data, time_scope)
    prob_output_file_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_prob_time/autonomous/simulated_log_{i}.csv"
    prob_df_with_time.to_csv(prob_output_file_path, index=False)

for j in log_l:
    petri_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_petri/autonomous/simulated_log_{j}.csv"
    petri_data = pd.read_csv(petri_path)
    train_split_data['timestamp'] = pd.to_datetime(train_split_data['timestamp'])
    first_resource = petri_data.iloc[0]['resource']
    resource_data = train_split_data[train_split_data['resource'] == first_resource]
    last_active_time = resource_data['timestamp'].max()
    petri_df_with_time = simulate_realistic_timestamps_gamma(petri_data, train_data, time_scope)
    petri_output_file_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_petri_time/autonomous/simulated_log_{j}.csv"
    petri_df_with_time.to_csv(petri_output_file_path, index=False)
'''

'\'\nimport pandas as pd\nimport numpy as np\nfrom collections import defaultdict\nfrom scipy.stats import gamma\n\ndef simulate_realistic_timestamps_gamma(events_df, train_data, agent_scope,\n                                        max_delta_seconds=43200):\n    """\n    Simulate timestamps using Gamma distribution for consecutive activity durations,\n    with robust safeguards to avoid invalid Gamma parameters and overflow.\n    \n    Parameters\n    ----------\n    events_df : pd.DataFrame\n        Must have \'case_id\', \'activity_name\', \'resource\'\n    train_data : pd.DataFrame\n        Must have \'case_id\', \'activity_type\', \'agent_id\', \'timestamp\'\n    agent_scope : dict\n        agent -> weekday -> (start_time, end_time)\n    max_delta_seconds : float\n        Maximum allowed duration in seconds to prevent overflow (default 12 hours)\n    \n    Returns\n    -------\n    pd.DataFrame with \'start_timestamp\' and \'end_timestamp\'\n    """\n    df = events_df.copy()\n   

In [68]:
import pandas as pd
import numpy as np
from scipy.stats import gamma

import pandas as pd
import numpy as np
from scipy.stats import gamma


def safe_gamma_rvs(shape, scale, max_seconds=86400):
    """
    Samples a gamma-distributed duration safely.
    Ensures: finite, positive, bounded.
    """
    if shape <= 0 or scale <= 0:
        return 60.0  # fallback 1 min

    delta = gamma.rvs(shape, scale=scale)

    # Invalid: NaN, inf, negative, zero
    if not np.isfinite(delta) or delta <= 0:
        return 60.0

    # Cap extreme values (avoid pandas overflow)
    if delta > max_seconds:
        delta = max_seconds

    return float(delta)


def align_to_agent_scope(ts, agent, agent_scope):
    """
    Align a timestamp to the closest valid working interval of the agent.
    agent_scope: {agent: {weekday: ("HH:MM:SS", "HH:MM:SS")}}
    """
    if agent not in agent_scope:
        return ts  # no constraint

    weekday = ts.weekday()

    while weekday not in agent_scope[agent]:
        ts += pd.Timedelta(days=1)
        weekday = ts.weekday()

    start_str, end_str = agent_scope[agent][weekday]
    sh, sm, ss = map(int, start_str.split(":"))
    eh, em, es = map(int, end_str.split(":"))

    start_time = ts.replace(hour=sh, minute=sm, second=ss)
    end_time   = ts.replace(hour=eh, minute=em, second=es)

    if ts < start_time:
        return start_time

    if ts > end_time:
        ts = (ts + pd.Timedelta(days=1)).replace(hour=sh, minute=sm, second=ss)
        return align_to_agent_scope(ts, agent, agent_scope)

    return ts


def simulate_with_case_and_agent_timing_scope(prob_data, train_data, agent_scope):
    """
    1. Model case-start intervals (gamma)
    2. Model agent-specific event-pair durations (gamma)
    3. Simulate timestamps respecting agent working hours
    4. Rename timestamp → start_timestamp + add end_timestamp = +10 min
    """

    # ======================================================
    # PART 1 — CASE START TIMING MODEL
    # ======================================================
    
    train_data["timestamp"] = pd.to_datetime(train_data["timestamp"], format='mixed')

    train_case_starts = (
        train_data.groupby("case_id")["timestamp"]
        .min()
        .sort_values()
    )

    deltas = train_case_starts.diff().dropna().dt.total_seconds()
    deltas = deltas[(deltas > 0) & (deltas < deltas.quantile(0.99))]

    mean = deltas.mean()
    var = deltas.var()

    shape_case = max(0.1, mean**2 / var)
    scale_case = max(0.1, var / mean)

    last_train_case_start = train_case_starts.max()  


    # ======================================================
    # PART 2 — AGENT EVENT-PAIR TIMING MODEL
    # ======================================================

    agent_pair_durations = {}
    train_sorted = train_data.sort_values(["case_id", "timestamp"])

    for agent, df_agent in train_sorted.groupby("agent"):  # resource = agent_id
        agent_pair_durations[agent] = {}

        for case, df_case in df_agent.groupby("case_id"):
            df_case = df_case.sort_values("timestamp")

            acts = df_case["activity_name"].tolist()
            times = df_case["timestamp"].tolist()

            for i in range(1, len(acts)):
                prev_act = acts[i - 1]
                act = acts[i]
                delta = (times[i] - times[i - 1]).total_seconds()

                if delta <= 0 or delta > 86400:
                    continue

                pair = (prev_act, act)
                agent_pair_durations[agent].setdefault(pair, []).append(delta)

    # Fit gamma distributions
    fitted_agent_stats = {}

    for agent in agent_pair_durations:
        fitted_agent_stats[agent] = {}

        for pair, vals in agent_pair_durations[agent].items():
            s = pd.Series(vals)
            if len(s) < 3:
                continue

            mean = s.mean()
            var = s.var()

            shape = max(0.1, mean**2 / var)
            scale = max(0.1, var / mean)

            fitted_agent_stats[agent][pair] = (shape, scale)


    # ======================================================
    # STEP 3 — SIMULATION
    # ======================================================

    df = prob_data.copy()
    # df = df.sort_values(["case_id"])
    df["start_timestamp"] = None

    unique_cases = df["case_id"].unique()
    case_start_map = {}

    ts_prev = last_train_case_start  # <-- predict next case after training data

    # --- simulate case starts ---
    for case in unique_cases:
        delta = safe_gamma_rvs(shape_case, scale_case)
        case_start = ts_prev + pd.Timedelta(seconds=delta)
        case_start_map[case] = case_start
        ts_prev = case_start


    # --- simulate events inside each case ---
    for case, df_case in df.groupby("case_id"):

        # Assign start of case
        first_idx = df_case.index[0]
        first_agent = df_case.at[first_idx, "resource"]

        ts = align_to_agent_scope(case_start_map[case], first_agent, agent_scope)
        df.at[first_idx, "start_timestamp"] = ts

        # Consecutive events
        for i in range(1, len(df_case)):
            idx_prev = df_case.index[i - 1]
            idx_curr = df_case.index[i]

            prev_ts = df.at[idx_prev, "start_timestamp"]
            prev_act = df_case.at[idx_prev, "activity_name"]
            curr_act = df_case.at[idx_curr, "activity_name"]
            agent = df_case.at[idx_curr, "resource"]

            pair = (prev_act, curr_act)

            if agent in fitted_agent_stats and pair in fitted_agent_stats[agent]:
                shape, scale = fitted_agent_stats[agent][pair]
                delta = safe_gamma_rvs(shape, scale)
            else:
                delta = 60  # fallback 1 minute

            new_ts = prev_ts + pd.Timedelta(seconds=delta)

            new_ts = align_to_agent_scope(new_ts, agent, agent_scope)

            df.at[idx_curr, "start_timestamp"] = new_ts

    # ======================================================
    # STEP 4 — Add end_timestamp
    # ======================================================

    df["end_timestamp"] = df["start_timestamp"] + pd.Timedelta(minutes=10)

    return df


log_l = [0,1,2,3,4,5,6,7,8,9]
train_split_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data/bpic_{year}_gt/autonomous/train_preprocessed.csv"
train_split_data = pd.read_csv(train_split_path)
for i in log_l:
    prob_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_prob/autonomous/simulated_log_{i}.csv"
    prob_data = pd.read_csv(prob_path)
    train_split_data['timestamp'] = pd.to_datetime(train_split_data['timestamp'])
    # first_resource = prob_data.iloc[0]['resource']
    # resource_data = train_split_data[train_split_data['resource'] == first_resource]
    # last_active_time = resource_data['timestamp'].max()
    prob_df_with_time = simulate_with_case_and_agent_timing_scope(prob_data, train_data, time_scope)
    prob_output_file_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_prob_time/autonomous/simulated_log_{i}.csv"
    prob_df_with_time.to_csv(prob_output_file_path, index=False)

for j in log_l:
    petri_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_petri/autonomous/simulated_log_{j}.csv"
    petri_data = pd.read_csv(petri_path)
    train_split_data['timestamp'] = pd.to_datetime(train_split_data['timestamp'])
    # first_resource = petri_data.iloc[0]['resource']
    # resource_data = train_split_data[train_split_data['resource'] == first_resource]
    # last_active_time = resource_data['timestamp'].max()
    petri_df_with_time = simulate_with_case_and_agent_timing_scope(petri_data, train_data, time_scope)
    petri_output_file_path = f"/Users/qingtan/Desktop/AgentSimulator-main/simulated_data_role_mining/BPIC{year}_petri_time/autonomous/simulated_log_{j}.csv"
    petri_df_with_time.to_csv(petri_output_file_path, index=False)